In [0]:
%fs ls 'abfss://gizmobox@deacoursestoragewestus22.dfs.core.windows.net/landing/ncell'

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox.bronze.ncell_data
AS SELECT * FROM read_files(
  'abfss://gizmobox@deacoursestoragewestus22.dfs.core.windows.net/landing/ncell',
  format => 'csv',
  header => true,
  inferSchema => true
)

In [0]:
%sql
Select * from gizmobox.bronze.ncell_data;

In [0]:
%sql
-- Convert date column from yyyyMMddHHmmss format to proper timestamp
ALTER TABLE gizmobox.bronze.ncell_data 
ADD COLUMN transaction_timestamp TIMESTAMP;

UPDATE gizmobox.bronze.ncell_data
SET transaction_timestamp = to_timestamp(CAST(date AS STRING), 'yyyyMMddHHmmss')

In [0]:
%sql
CREATE OR REPLACE TABLE gizmobox.silver.ncell_recharge_daily_agg AS
SELECT 
  retailer_id,
  DATE(transaction_timestamp) AS transaction_date,
  COUNT(*) AS total_transactions,
  SUM(abs(transaction_amount)) AS total_amount,
  AVG(abs(transaction_amount)) AS avg_amount,
  MIN(abs(transaction_amount)) AS min_amount,
  MAX(abs(transaction_amount)) AS max_amount
FROM gizmobox.bronze.ncell_data
WHERE transaction_type = 'Recharge'
  AND transaction_timestamp IS NOT NULL
GROUP BY retailer_id, DATE(transaction_timestamp)
ORDER BY retailer_id, transaction_date

In [0]:
%sql
Select * from gizmobox.silver.ncell_recharge_daily_agg;

In [0]:
%sql
Select * from gizmobox.gold.ncell_forecast_total_amount;